In [18]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import sys
sys.path.append("../")
import housinginsights
from housinginsights.ingestion import ManifestReader, DataReader
from housinginsights.ingestion import ACSRentCleaner, GenericCleaner

import json


ACS cleaner loaded


In [37]:
#copied from scripts
def load_meta_data(filename='meta.json'):
    """
    Expected meta data format:
        { tablename: {fields:[
            {   "display_name": "Preservation Catalog ID",
                "display_text": "description of what this field is",
                "source_name": "Nlihc_id",
                "sql_name": "nlihc_id",
                "type": "object"
            }
            ]}
        }
    """
    with open(filename) as fh:
        meta = json.load(fh)

    json_is_valid = True
    try:
        for table in meta:
            for field in meta[table]['fields']:
                for key in field:
                    if key not in ('display_name', 'display_text', 'source_name', 'sql_name', 'type'):
                        json_is_valid = False
                        first_json_error = "Location: table: {}, section: {}, attribute: {}".format(table, field, key)
                        raise ValueError("Error found in JSON, check expected format. {}".format(first_json_error))
    except:
        raise ValueError("Error found in JSON, check expected format.")

    return meta


In [38]:
meta = load_meta_data('meta_sample.json')
manifest = ManifestReader('manifest_sample.csv')

In [43]:
for manifest_row in manifest:
    csv_reader = DataReader(manifest_row=manifest_row)
    
    #Assign an appropriate testing cleaner
    if manifest_row['destination_table'] == "acs_rent_median":
        cleaner = ACSRentCleaner(meta, manifest_row)
    else:
        cleaner = GenericCleaner(meta,manifest_row)
        
    for data_row in csv_reader:
        print(data_row)
        clean_data_row = cleaner.clean(data_row)
        print(clean_data_row)

{'Geo2010': 'Tract 49.02', 'Proj_image_url': 'http://citizenatlas.dc.gov/mobilevideo/20040926/QQ112940.jpg', 'Proj_streetview_url': 'http://maps.google.com/maps?z=16&layer=c&cbll=38.90811789,-77.0221966&cbp=11,268.61496892357,,0,2.09', 'Proj_City': 'Washington', 'Bldg_count': '1', 'Hud_Mgr_Name': 'Edgewood Management Corporation', 'Subsidy_End_First': '3/1/2020', 'Proj_address_id': '238401', 'Ward2012': 'Ward 6', 'Cat_At_Risk': 'No', 'Hud_Own_Name': '1330 7th Street Limited Partnership', 'Update_Dtm': '01MAR16:15:34:26', 'Nlihc_id': 'NL000001', 'Cat_Lost': 'No', 'Proj_ST': 'DC', 'contract_number': 'DC39M000025', 'Zip': 'ZIP 20001', 'Cluster_tr2000_name': 'Shaw, Logan Circle', 'Category_Code': '5 - Other Subsidized Property', 'Psa2012': 'PSA 308', 'Proj_x': '398074.8', 'Proj_y': '137806.65', 'Proj_lat': '38.90811789', 'Subsidy_Start_First': '1/1/2004', 'Cat_Failing_Insp': 'No', 'Hud_Own_Effect_dt': '1/30/2004', 'Hud_Own_Type': 'Limited dividend', 'Proj_Addre': '1330 7th Street NW', 'Cat